In [1]:
import cv2
import numpy as np
import torch
from ultralytics import YOLO
import requests
import base64
import json
from segment_anything import sam_model_registry, SamPredictor

Creating new Ultralytics Settings v0.0.6 file  
View Ultralytics Settings with 'yolo settings' or at 'C:\Users\hp\AppData\Roaming\Ultralytics\settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [4]:
# Load YOLOv8 model for object detection
yolo_model = YOLO("yolov8m.pt")  # Medium model for better accuracy

# Load Segment Anything Model (SAM) for segmentation
sam = sam_model_registry["vit_b"](checkpoint="sam_vit_b.pth")
sam_predictor = SamPredictor(sam)
sam.to("cuda" if torch.cuda.is_available() else "cpu")

# Function to encode image to base64
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")

# Load input image
image_path = "input.jpg"
image = cv2.imread(image_path)

# Run YOLO to detect objects
results = yolo_model(image)

# Generate mask for detected objects
mask = np.zeros(image.shape[:2], dtype=np.uint8)
for result in results:
    for box in result.boxes.xyxy:  # Get bounding box
        x1, y1, x2, y2 = map(int, box)
        mask[y1:y2, x1:x2] = 255  # Create white mask for object region

# Save mask
mask_path = "mask.png"
cv2.imwrite(mask_path, mask)

# Send image and mask to LaMa Cleaner API
image_base64 = encode_image(image_path)
mask_base64 = encode_image(mask_path)

lama_url = "http://127.0.0.1:8080/inpaint"
payload = {
    "image": image_base64,
    "mask": mask_base64,
    "model": "lama",
    "device": "cuda"
}

response = requests.post(lama_url, json=payload)

# Save the output image
if response.status_code == 200:
    output_data = json.loads(response.text)
    output_image = base64.b64decode(output_data["output"])
    
    with open("output.jpg", "wb") as out_file:
        out_file.write(output_image)
    
    print("✅ Object removed successfully! Check output.jpg")
else:
    print("❌ Error:", response.text)


RuntimeError: Error(s) in loading state_dict for Sam:
	Unexpected key(s) in state_dict: "image_encoder.blocks.12.norm1.weight", "image_encoder.blocks.12.norm1.bias", "image_encoder.blocks.12.attn.rel_pos_h", "image_encoder.blocks.12.attn.rel_pos_w", "image_encoder.blocks.12.attn.qkv.weight", "image_encoder.blocks.12.attn.qkv.bias", "image_encoder.blocks.12.attn.proj.weight", "image_encoder.blocks.12.attn.proj.bias", "image_encoder.blocks.12.norm2.weight", "image_encoder.blocks.12.norm2.bias", "image_encoder.blocks.12.mlp.lin1.weight", "image_encoder.blocks.12.mlp.lin1.bias", "image_encoder.blocks.12.mlp.lin2.weight", "image_encoder.blocks.12.mlp.lin2.bias", "image_encoder.blocks.13.norm1.weight", "image_encoder.blocks.13.norm1.bias", "image_encoder.blocks.13.attn.rel_pos_h", "image_encoder.blocks.13.attn.rel_pos_w", "image_encoder.blocks.13.attn.qkv.weight", "image_encoder.blocks.13.attn.qkv.bias", "image_encoder.blocks.13.attn.proj.weight", "image_encoder.blocks.13.attn.proj.bias", "i